In [1]:
PROJECT_ID = "genie-ui-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["PG_HOST"] = ""

In [2]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [3]:
import tempfile
import os
from typing import List, Optional, Tuple, Dict
from common.utils.logging_handler import Logger
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)
from common.utils.errors import (ResourceNotFoundException,
                                 ValidationError)
from common.utils.http_exceptions import InternalServerError
from utils.errors import NoDocumentsIndexedException
from google.cloud import storage
from services import llm_generate, embeddings
from services.query import query_prompts
from services.query.vector_store import VectorStore
from services.query.data_source import DataSource
    
from config import (PROJECT_ID, DEFAULT_QUERY_CHAT_MODEL,
                        DEFAULT_QUERY_EMBEDDING_MODEL)
import spacy

INFO: [config/config.py:61 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/model_config.py:341 - set_model_config()] Provider [Vertex] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [VertexAI-Text] enablement status [True]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Vertex] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [VertexAI-Chat] enablement status [True]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Vertex] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [VertexAI-Chat-Palm2] enablement status [True]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Vertex] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [VertexAI-Chat-Palm2-V2] enablement status [True]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Vertex] ena

/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/model_config.py:341 - set_model_config()] Provider [Vertex] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [VertexAI-Chat-Palm2-32k] enablement status [True]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Langchain] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [VertexAI-Chat-Palm2-32k-Langchain] enablement status [True]


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/model_config.py:341 - set_model_config()] Provider [Langchain] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [VertexAI-Chat-Gemini-Pro-Langchain] enablement status [True]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Truss] enablement status [False]
INFO: [config/model_config.py:361 - set_model_config()] Model [Truss-Llama2-Chat] enablement status [False]
INFO: [config/model_config.py:341 - set_model_config()] Provider [vLLM] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [vLLM-Gemma-Chat] enablement status [False]
INFO: [config/model_config.py:341 - set_model_config()] Provider [ModelGarden] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [VertexAI-ModelGarden-LLAMA2-Chat] enablement status [False]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Langchain] enablement status [True]


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/model_config.py:361 - set_model_config()] Model [OpenAI-GPT4] enablement status [False]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Langchain] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [OpenAI-GPT4-latest] enablement status [False]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Langchain] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [OpenAI-GPT3.5] enablement status [False]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Langchain] enablement status [True]
INFO: [config/model_config.py:361 - set_model_config()] Model [Cohere] enablement status [False]
INFO: [config/model_config.py:341 - set_model_config()] Provider [LLMService] enablement status [False]
INFO: [config/model_config.py:361 - set_model_config()] Model [Llama2cpp] enablement status [False]
INFO: [config/model_config.py:341 - set_model_config()] Provider [Vertex] enab

/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:343: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/config.py:112 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:113 - <module>()] ENABLE_OPENAI_LLM = False
INFO: [config/config.py:114 - <module>()] ENABLE_COHERE_LLM = False
INFO: [config/config.py:115 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:116 - <module>()] ENABLE_TRUSS_LLAMA2 = False
INFO: [config/config.py:117 - <module>()] ENABLE_VLLM_GEMMA = True
INFO: [config/config.py:180 - <module>()] Loaded default manifest from /Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/../src/config/document_manifest.json
INFO: [config/vector_store_config.py:44 - <module>()] PG_HOST = [None]
INFO: [config/vector_store_config.py:45 - <module>()] PG_DBNAME = [pgvector]
INFO: [config/vector_store_config.py:74 - <module>()] PG_HOST is set to [None], not connecting to pgvector
INFO: [config/vector_store_config.py:81 - <module>()] Default vector store = [matching_engine]
INFO: [config/onedrive_config.py:

In [6]:
# do this once per vm environment
!python -m spacy download en_core_web_md

nlp = spacy.load("en_core_web_md")
storage_client = storage.Client(project=PROJECT_ID)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [7]:
qe_list = QueryEngine.fetch_all()
[(qe.name, qe.id) for qe in qe_list]

[('Test QEAdmin Engine', '8wSqUJpSOXkwYyH8toH8'),
 ('NY Times', 'gdjbNCJaTFBI9VOQ5wea'),
 ('Eagle County Rental Assistance Program Integrated Search',
  'bL8G7DTKs9mtf0FLkj5F'),
 ('Eagle County Residential Assistance Docs', 'rfdRfI2CF0o6PFzoERDH'),
 ('Eagle County Residential Assistance 2', 'zigHiOwtcmEYGEgHEPu9'),
 ('Eagle County Residential Assistance 3', 'eWxGPu2xDBZnYPDwez6Z'),
 ('Eagle County Residential Assistance', 'EXGzHlzbnRiE8zqfdQpp'),
 ('Princeton Web Site', 'jWgVRunaEkT85n6alm9E'),
 ('NYC DOE Site', 'my77xrKcW5s21MLk3RtS'),
 ('Covered CA Integrated', 'QDurLC0rIwbZRjcuH7jf'),
 ('Covered CA Docs', 'WFmXMhSQZpU2EQjGOwyd'),
 ('Covered CA Genie', '7ijUSBdrpyo6ldhFTQgs')]

In [ ]:
query_engine = "lukman-test-vs-6"
doc_url = "gs://lramsey-dev-mira-demo-docs"
user_id = "epKVwPZeIBz1rrqcDm1B"
is_public=True
query_description="test vertex search"

In [ ]:
#query_engine = "lukman-test-html"
#doc_url = "https://health.ny.gov/health_care/medicaid/rates/manual/apg_provider_manual_december.htm"
#user_id = "epKVwPZeIBz1rrqcDm1B"
#query_description="test depth limit"
#storage_client = storage.Client(project=PROJECT_ID)

In [ ]:
from common.models.llm_query import QE_TYPE_VERTEX_SEARCH, QE_TYPE_LLM_SERVICE

In [ ]:
# create q_engine
params = {"depth_limit": 1}
query_engine_type = QE_TYPE_VERTEX_SEARCH
embedding_type = DEFAULT_QUERY_EMBEDDING_MODEL
llm_type = DEFAULT_QUERY_EMBEDDING_MODEL
vector_store_type = "langchain_pgvector"
q_engine = QueryEngine(name=query_engine,
                     created_by=user_id,
                     query_engine_type=QE_TYPE_VERTEX_SEARCH,
                     llm_type=llm_type,
                     description=query_description,
                     embedding_type=embedding_type,
                     vector_store=vector_store_type,
                     params=params,
                     doc_url=doc_url)
q_engine.save()

In [ ]:
from google.cloud import discoveryengine_v1alpha as discoveryengine

In [ ]:
location = "global"
project_id = PROJECT_ID

In [ ]:
parent = "projects/lramsey-dev/locations/global/collections/default_collection"

In [ ]:
# Create a Data Store...
data_store_id = q_engine.name

dss_client = discoveryengine.DataStoreServiceClient()

ds_parent = parent

content_config = discoveryengine.DataStore.ContentConfig.CONTENT_REQUIRED
data_store = discoveryengine.DataStore(display_name=q_engine.name,
                                       industry_vertical="GENERIC",
                                       solution_types=["SOLUTION_TYPE_SEARCH"],
                                       content_config=content_config)

In [ ]:
data_store

In [ ]:
ds_request = discoveryengine.CreateDataStoreRequest(parent=ds_parent,
                                                    data_store_id=data_store_id,
                                                    data_store=data_store)

operation = dss_client.create_data_store(request=ds_request)


In [ ]:
operation.done()

In [ ]:
gcs_uri = doc_url

ds_client = discoveryengine.DocumentServiceClient()

parent = ds_client.branch_path(
  project=project_id,
  location=location,
  data_store=data_store_id,
  branch="default_branch",
)

gcs_uris = [
  f"{gcs_uri}/*.pdf",
#  f"{gcs_uri}/*.html",
]

request = discoveryengine.ImportDocumentsRequest(
  parent=parent,
  gcs_source=discoveryengine.GcsSource(
      input_uris=gcs_uris, data_schema="content"
  ),
  # Options: `FULL`, `INCREMENTAL`
  reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
)

# Make the request
operation = ds_client.import_documents(request=request)

In [ ]:
response = operation.result()

In [ ]:
response

In [ ]:
metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

In [ ]:
metadata

In [ ]:
parent = f"projects/{project_id}/locations/global/collections/default_collection"
engine = discoveryengine.Engine()
engine.display_name = q_engine.name
engine.solution_type = "SOLUTION_TYPE_SEARCH"
engine.data_store_ids = [data_store_id]
request = discoveryengine.CreateEngineRequest(parent=parent,
                                            engine=engine,
                                            engine_id=data_store_id)

In [ ]:
es_client = discoveryengine.EngineServiceClient()
operation = es_client.create_engine(request=request)

In [ ]:
operation.done()

In [ ]:
from services.query.vertex_search import create_client, import_documents_gcs

In [ ]:
client,parent = create_client(PROJECT_ID, location, data_store_id)

In [ ]:
parent

In [ ]:
operation = import_documents_gcs(doc_url, client, parent)

In [ ]:
from services import llm_generate, embeddings
from services.query import query_prompts
from services.query.vector_store import VectorStore
from services.query.data_source import DataSource
from services.query.query_service import datasource_from_url
from config import (PROJECT_ID, DEFAULT_QUERY_CHAT_MODEL,
                    DEFAULT_QUERY_EMBEDDING_MODEL)
import spacy

In [ ]:
doc_name, index_doc_url, doc_filepath = doc_filepaths[0]

In [ ]:
text_chunks = data_source.chunk_document(doc_name, index_doc_url, doc_filepath)

In [ ]:
len(text_chunks)

In [ ]:
text_chunks[8]

In [ ]:
data_source.text_to_sentence_list(text_chunks[6])

In [ ]:
embeddings.get_embeddings(text_chunks, embedding_type)

In [ ]:
from utils.html_helper import html_to_text, html_to_sentence_list
query_description = "Policies and guidance on billing for dental procedures using CDT codes for Federally Qualified Health Centers (FQHC).  Includes information on state agency reporting of billing against current dental terminology (CDT) codes.  Also includes information on billing, coding and other guidelines that support the implementation of the CY 2023 Medicare Physician Fee Schedule Final Rule on Dental Services."

In [ ]:
from services.query.query_service import process_documents, vector_store_from_query_engine

vector_store = vector_store_from_query_engine(q_engine)

In [ ]:
text_chunks

In [ ]:
from w3lib.html import replace_escape_chars
import re

def clean_text(text):
  # Replace specific unprocessable characters
  cleaned_text = text.replace("\x00", "")

  # replace escape characters
  cleaned_text = replace_escape_chars(cleaned_text)

  # remove all non-printable characters
  cleaned_text = re.sub(r"[^\x20-\x7E]", "", cleaned_text)

  return cleaned_text

In [ ]:
text_chunks = [clean_text(x) for x in text_chunks]

In [ ]:
text_chunks

In [ ]:
QueryEngine.delete_by_id(q_engine.id)

In [ ]:
for i in range(0, len(text_chunks)):
  clean_text = html_to_text(text_chunks[i])
  sentences = html_to_sentence_list(text_chunks[i])
  query_doc_chunk = QueryDocumentChunk(
                        query_engine_id=q_engine.id,
                        query_document_id=query_doc.id,
                        index=i+index_base,
                        text=text_chunks[i],
                        clean_text=clean_text,
                        sentences=sentences)

In [ ]:
storage_client = storage.Client(project=PROJECT_ID)

vector_store = VectorStore(q_engine)

In [ ]:
docs_processed

In [ ]:
docs_not_processed